In [33]:
import pandas as pd
import numpy as np
import requests
import json

In [127]:
df_parks = pd.read_csv('Parks/Parks.csv')
df_parks = df_parks.fillna('')
df_parks

,Park_Name,Park_Addre,Park_Zip,Park_State,Park_Count,Park_Place
0,Hampton Hills Park,1285 Bath Rd,44223.0,Ohio,Summit County,Cuyahoga Falls city
1,Frank Boulevard Park,2086 Stockbridge Rd,44313.0,Ohio,Summit County,Akron city
2,Fairlawn Heights Park,Fairlawn Blvd,44313.0,Ohio,Summit County,Akron city
3,Northwest Park,1730 Shatto Ave,44313.0,Ohio,Summit County,Akron city
4,Westgate Manor Park,W Market St,44313.0,Ohio,Summit County,Akron city
...,...,...,...,...,...,...
129797,RiverPark Pool & Park,,77479.0,Texas,Fort Bend County,Sugar Land city
129798,Searls Pool & Park,,77478.0,Texas,Fort Bend County,Sugar Land city
129799,The Lakes Park,,77479.0,Texas,Fort Bend County,Sugar Land city
129800,Sugar Lakes - The Landing Park,,77478.0,Texas,Fort Bend County,Sugar Land city


In [128]:
df_parks.columns

Index(['Park_Name', 'Park_Addre', 'Park_Zip', 'Park_State', 'Park_Count',
       'Park_Place'],
      dtype='object')

In [169]:
addresses = df_parks[['Park_Addre', 'Park_Zip', 'Park_State', 'Park_Count']].to_numpy()
addresses.shape

(129802, 4)

In [170]:
names = df_parks['Park_Name']
names.shape

(129802,)

In [208]:
def get_lat_long(query):
    response = requests.get(query)
    
    lat = ''
    lng = ''
    
    try:
        response = response.json()
    except:
        print("Error when running response.json()\n",query)
        return lat, lng 
    
    if(response):
        lat = float(response[0]['lat'])
        lng = float(response[0]['lon'])    
        
    return lat, lng

In [216]:
def get_all_lat_long(names, addresses):
    url = "https://nominatim.openstreetmap.org/search?"

    latitudes = np.array([])
    longitudes = np.array([])
    for i in range(len(addresses)):

        query_array=[]
        # address
        if(addresses[i][0] != ''):
            query_array.append("street=" + addresses[i][0].replace(' ','+'))

        # zip
        if(addresses[i][1] != ''):
            query_array.append("postalcode=" + str(int(addresses[i][1])))

        # state
        if(addresses[i][2] != ''):
            query_array.append("state=" + addresses[i][2].replace(' ','+'))

        # county
        if(addresses[i][3] != ''):
            query_array.append("county=" + addresses[i][3].replace(' ','+'))

        query_array.append("country=United+States")
        query_array.append("limit=1")

        query = url + '&'.join(query_array,) + "&format=json"
        
        lat, lng = get_lat_long(query)
        
        print(i, "complete...", end='\r')

        latitudes = np.append(latitudes, lat)
        longitudes = np.append(longitudes, lng)
                    
    df_lat = pd.DataFrame(latitudes)
    df_lon = pd.DataFrame(longitudes)
    
    df_all = pd.concat([names,df_lat,df_lon],axis=1)
    df_all.columns=['Name','lat','lng']
    return df_all
        

In [217]:
df_lat_lng = get_all_lat_long(names, addresses)
df_lat_lng

Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=#+1+Poydras&postalcode=70130&state=Louisiana&county=Orleans+Parish&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=3103+E+BANBURY+RD+#+APXBT&postalcode=84121&state=Utah&county=Salt+Lake+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=507+Florida+Ave+#204&postalcode=20170&state=Virginia&county=Fairfax+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=8645+Marhis+Ave+#102&postalcode=22079&state=Virginia&county=Fairfax+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=Kenmore+Drive+and+Charleston+Terrace,+#1+Cul+De+Sac&postalcode=20007&state=District+of+Columbia&county=Dist

Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=3200+RIVERSIDE+DR+#+PARK&postalcode=33065&state=Florida&county=Broward+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=3830+CORAL+SPRINGS+DR+#+PARK&postalcode=33065&state=Florida&county=Broward+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=8537+NW+38TH+DR+#+PARK&postalcode=33065&state=Florida&county=Broward+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=11900+NW+30TH+ST+#+PARK&postalcode=33065&state=Florida&county=Broward+County&country=United+States&limit=1&format=json
Error when running response.json()
 https://nominatim.openstreetmap.org/search?street=10284+SW+1ST+CT+#+PARK&postalcode=33321&state=Florida&county=Broward+County&country=Unite

,Name,lat,lng
0,Hampton Hills Park,41.16705360433309,-81.55086971230959
1,Frank Boulevard Park,41.10977022746043,-81.59050331127713
2,Fairlawn Heights Park,41.112358,-81.592631
3,Northwest Park,41.11585335,-81.57492348837991
4,Westgate Manor Park,41.1108087,-81.5719983
...,...,...,...
129797,RiverPark Pool & Park,29.511218,-95.7807348
129798,Searls Pool & Park,29.511218,-95.7807348
129799,The Lakes Park,29.511218,-95.7807348
129800,Sugar Lakes - The Landing Park,29.511218,-95.7807348


In [218]:
df_lat_lng.to_csv('Parks/Parks_Condensed.csv',index=False)

In [230]:
df_lat_lng.shape

(129802, 3)

In [267]:
index_names = df_lat_lng[df_lat_lng['lat']==''].index

df_lat_lng_drop = df_lat_lng.drop(index_names)
df_lat_lng_drop.shape

(119042, 3)

In [268]:
index_names = df_lat_lng_drop[df_lat_lng_drop['Name']==''].index

df_lat_lng_drop_2 = df_lat_lng_drop.drop(index_names)
df_lat_lng_drop_2.shape

(119036, 3)

In [269]:
df_lat_lng_drop_2.to_csv('Parks/Parks_Final.csv', index=False)

In [274]:
array_to_fix = df_lat_lng_drop_2.to_numpy()

for i in range(len(array_to_fix)):
    array_to_fix[i][0] = array_to_fix[i][0].replace('\r',' ')
    array_to_fix[i][0] = array_to_fix[i][0].replace('\n',' ')
    array_to_fix[i][1] = round(float(array_to_fix[i][1]),9)
    array_to_fix[i][2] = round(float(array_to_fix[i][2]),9)

In [275]:
array_to_fix[2400:2415]

array([['Mckeithen Park', 41.0598483, -73.5181397],
       ['Rippowam Park  Rippowam Park  ', 41.0519069, -73.5421666],
       ['Stamford Museum And Nature Center', 41.1260638, -73.5455438],
       ["St. John'S Park", 41.2008607, -73.1163423],
       ['Veterans Memorial Park', 41.0498019, -73.5395765],
       ['Finch Meadow / Levine Site', 41.20353095, -73.416568015],
       ['Meadowpark Open Space #1', 41.0923959, -73.5538921],
       ['Meadowpark Open Space #2', 41.0922582, -73.5530198],
       ['Meadowpark Open Space #3', 41.0917453, -73.5519058],
       ['Cove Island Park', 41.049069742, -73.504900733],
       ['Depreta Park', 41.049003, -73.514721],
       ['Mill River Park', 41.0540582, -73.5462758],
       ['Woodway Park', 41.1027594, -73.508353],
       ['George T. Barrett Field', 41.0761306, -73.532656],
       ['Scalzi Park', 41.151578, -73.1284142]], dtype=object)

In [276]:
df_final = pd.DataFrame(array_to_fix, columns=['Park_Name','lat','lng'])
df_final.to_csv('Parks/Parks_Final.csv', index=False)

In [277]:
df_final

,Park_Name,lat,lng
0,Hampton Hills Park,41.167054,-81.55087
1,Frank Boulevard Park,41.10977,-81.590503
2,Fairlawn Heights Park,41.112358,-81.592631
3,Northwest Park,41.115853,-81.574923
4,Westgate Manor Park,41.110809,-81.571998
...,...,...,...
119031,RiverPark Pool & Park,29.511218,-95.780735
119032,Searls Pool & Park,29.511218,-95.780735
119033,The Lakes Park,29.511218,-95.780735
119034,Sugar Lakes - The Landing Park,29.511218,-95.780735
